In [67]:
import pandas as pd
import numpy as np

In [68]:
df_micro = pd.DataFrame(
    {
        "ni_transito": [
            10000,
            100000,
            1000000,
            10000000,
            100000000,
            1000000000,
        ]
    }
)

In [69]:
df_micro

,ni_transito
0,10000
1,100000
2,1000000
3,10000000
4,100000000
5,1000000000


# FORMULAS FIJAS DEL MODELO

In [70]:
modulo_rigidez_MPa = 4500
espesor_mezcla_asfaltica_cm = 15
numero_ciclos_carga = 134

In [71]:
rigidez_convertida = 145 * modulo_rigidez_MPa
rigidez_convertida


652500

In [72]:
espesor_mezcla_pulgadas = espesor_mezcla_asfaltica_cm / 2.54
espesor_mezcla_asfaltica_cm

15

In [73]:
factor_ajuste_modelo = 10 ** (4.84 * (11 / (11 + 5) - 0.69))
factor_ajuste_modelo

0.9725232662363479

In [74]:
factor_ajuste_resistencia = float(1 / (0.000398 + (0.003602 / (1 + np.exp(11.02 - 3.49 * espesor_mezcla_pulgadas)))))
factor_ajuste_resistencia

250.01539692873743

# Tabla de Calculo de indices de fatiga - tabla derecha

In [75]:
# COPIA DF micro a  calculos base fatiga
df_calculos_base_fatiga = df_micro.copy()

In [76]:
df_calculos_base_fatiga

,ni_transito
0,10000
1,100000
2,1000000
3,10000000
4,100000000
5,1000000000


In [77]:
# Calculo SPDM DERECHO 
df_calculos_base_fatiga['SPDM'] = (1000000 * (df_calculos_base_fatiga['ni_transito'] * 1/10 * (0.856 * 11 + 1.08)**-5 * (modulo_rigidez_MPa * 1000000)**1.8)**(-1/5))

In [78]:
# Calculo de AI MS-1
df_calculos_base_fatiga['AI MS-1'] = 1000000 * ((df_calculos_base_fatiga['ni_transito'] * rigidez_convertida**0.854) / (18.4 * 0.00432 * factor_ajuste_modelo))**(-1 / 3.291)

In [79]:
# Calculo de Cedex-Shell
df_calculos_base_fatiga['Cedex-Shell'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.00000000906)**(-1 / 3.6706)

In [80]:
# Calculo de Illinois
df_calculos_base_fatiga['Illinois'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.000005)**(-1 / 3)

In [81]:
# Calculo de Minnesota
df_calculos_base_fatiga['Minnesota'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.00000283)**(-1 / 3.206)

In [82]:
# Calculo de USACE
df_calculos_base_fatiga['USACE'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 478.63 * (145 * modulo_rigidez_MPa)**2.66)**(-1 / 5)

In [83]:
#Calculo de ME-PDG
df_calculos_base_fatiga['ME-PDG'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] * rigidez_convertida**1.281 / (18.4 * 0.00432 * factor_ajuste_modelo * 0.007566 * factor_ajuste_resistencia))**(-1 / 3.9492)

In [84]:
# #Calculo de AUSTROADS
df_calculos_base_fatiga['AUSTROADS'] = ((df_calculos_base_fatiga['ni_transito'] / 2)**-0.2 * 6918 * (0.856 * 11 + 1.08) / (modulo_rigidez_MPa**0.36))

In [85]:
df_calculos_base_fatiga

,ni_transito,SPDM,AI MS-1,Cedex-Shell,Illinois,Minnesota,USACE,ME-PDG,AUSTROADS
0,10000,882.810149,866.714311,523.720928,793.700526,1051.398728,439.180601,775.599737,639.814199
1,100000,557.015547,430.543422,279.682250,368.403150,512.686980,277.104226,432.934118,403.695468
2,1000000,351.453050,213.873979,149.358478,170.997595,249.998342,174.840946,241.660668,254.714620
3,10000000,221.751883,106.242661,79.761783,79.370053,121.905127,110.317179,134.893223,160.714061
4,100000000,139.915979,52.776420,42.595119,36.840315,59.443834,69.605434,75.296413,101.403717
5,1000000000,88.281015,26.216875,22.747036,17.099759,28.986225,43.918060,42.029909,63.981420


# Tabla de Calculo de indices de fatiga - tabla izquierda